In [1]:
%matplotlib inline

import pandas as pd
import json
import numpy as np
from pandas.io.json import json_normalize
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

In [2]:
#Read in CSV data files

airbnb = pd.read_csv('zip_clean.csv',index_col=0)
bydz_availcount = pd.read_csv('bydz_availcount.csv',header=None)
bydz_availcount.columns=['date_zip','avail_count']
bydzrt_roomcount = pd.read_csv('bydzrt_roomcount.csv',header=None)
bydzrt_roomcount.columns=['date_zip','room_type','date','bydzrt_count']
prr_df = pd.read_csv('prr_df.csv',index_col=0)
mlp_df = pd.read_csv('mlp_df.csv',index_col=0)
air_zillow = pd.read_csv('airzillow_df.csv', index_col=0)

In [3]:
data = pd.read_csv('Gaz_zcta_national.txt', sep="\t")
data.columns = ["zcta", "pop10", "hu10", "aland","awater","aland_sqmi","awater_sqmi","lat","long"]
data.head()


,zcta,pop10,hu10,aland,awater,aland_sqmi,awater_sqmi,lat,long
0,601,18570,7744,166659789,799296,64.348,0.309,18.180555,-66.749961
1,602,41520,18073,79288158,4446273,30.613,1.717,18.362268,-67.176130
2,603,54689,25653,81880442,183425,31.614,0.071,18.455183,-67.119887
3,606,6615,2877,109580061,12487,42.309,0.005,18.158345,-66.932911
4,610,29016,12618,93021467,4172001,35.916,1.611,18.290955,-67.125868


In [4]:
airbnb15 = airbnb.loc[airbnb['year'] == 2015]
airbnb15.to_csv('airbnb15.csv')
airbnb16 = airbnb.loc[airbnb['year'] == 2016]
airbnb16.to_csv('airbnb16.csv')
airbnb17 = airbnb.loc[airbnb['year'] == 2017]
airbnb17.to_csv('airbnb17.csv')

In [5]:
airbnb17_count = airbnb17[['id','date_zip','zipcode','month']].groupby(['date_zip','zipcode','month'], as_index=False).agg('count')
airbnb17_count = pd.merge(airbnb17_count, data[['zcta','pop10','lat','long']],how='left', left_on='zipcode', right_on='zcta')
airbnb17_count = airbnb17_count[['date_zip','zipcode','month','id','pop10','lat','long']]
airbnb17_count = airbnb17_count.rename(columns={'id': 'airbnb_count'})
airbnb17_count['norm_count'] = airbnb17_count['airbnb_count'].div(airbnb17_count['pop10'])
nullpop = airbnb17_count.loc[airbnb17_count['pop10']==0][['date_zip','zipcode','month','airbnb_count','pop10','lat','long']]
nullpop
airbnb17_count = airbnb17_count.loc[airbnb17_count['pop10']!=0]
airbnb17_count
airbnb17_count.to_csv('airbnb17_count.csv')

# print(type(airbnb17_count['zipcode']))
# print(type(data['zcta']))

In [6]:
airbnb17_countavg = airbnb17_count[['zipcode','airbnb_count','pop10','norm_count']].groupby(['zipcode'],as_index=False).agg('mean')
airbnb17_countavg.to_csv('airbnb17_countavg.csv')

In [7]:
sortnorm_17 = airbnb17_countavg.sort_values(by='norm_count',ascending=False)
topnorm_17 = sortnorm_17.loc[sortnorm_17['norm_count']>=0.01]
sortnorm_17

,zipcode,airbnb_count,pop10,norm_count
15,10018,179.600,5229.0,0.034347
10,10012,594.700,24090.0,0.024687
32,10036,598.300,24711.0,0.024212
106,11222,710.800,36934.0,0.019245
12,10014,574.300,31959.0,0.017970
121,11238,880.100,49262.0,0.017866
95,11211,1592.000,90117.0,0.017666
16,10019,725.800,42870.0,0.016930
100,11216,898.300,54316.0,0.016538
0,10001,338.800,21102.0,0.016055


In [8]:
mlp_avg = mlp_df[['median_list_price','year','zip']].groupby(['zip','year'], as_index=False).agg('mean')
mlp_avg17 = mlp_avg[['median_list_price','year','zip']].loc[mlp_avg['year']==2017]
mlp_avg17

,median_list_price,year,zip
1,3.030000e+06,2017,10001
5,1.588250e+06,2017,10002
8,2.546750e+06,2017,10003
11,1.150167e+06,2017,10004
20,1.439875e+06,2017,10005
23,2.747083e+06,2017,10006
26,4.716831e+06,2017,10007
29,2.424167e+06,2017,10010
34,2.647498e+06,2017,10011
37,4.394583e+06,2017,10012


In [3]:
airbnb17.head()

NameError: name 'airbnb17' is not defined

In [9]:
airbnb17_price = airbnb17[['zipcode','price']].groupby(['zipcode'], as_index=False).agg('mean')

In [10]:
topzip17 = pd.merge(topnorm_17,airbnb17_price,how='left',on='zipcode')
# topzip17 = topzip17.dropna(axis=0,how='any')
topzip17 = topzip17[['zipcode','norm_count','price']]
topzip17.to_csv('topzip17.csv')
topzip17

,zipcode,norm_count,price
0,10018,0.034347,229.126949
1,10012,0.024687,224.851354
2,10036,0.024212,208.302691
3,11222,0.019245,120.881401
4,10014,0.017970,227.079401
5,11238,0.017866,130.694239
6,11211,0.017666,146.601759
7,10019,0.016930,200.083219
8,11216,0.016538,107.068129
9,10001,0.016055,213.537485


In [11]:
topnormprice_17 = topzip17.sort_values(by='price',ascending=False)
topnormprice_17

,zipcode,norm_count,price
0,10018,0.034347,229.126949
4,10014,0.017970,227.079401
1,10012,0.024687,224.851354
20,10013,0.011996,223.137526
14,10011,0.013283,217.563940
9,10001,0.016055,213.537485
10,10006,0.015277,209.171739
2,10036,0.024212,208.302691
12,10003,0.014672,200.157908
7,10019,0.016930,200.083219


In [12]:
sortnorm_17

norm_17 = pd.merge(sortnorm_17,airbnb17_price,how='left',on='zipcode')
# topzip17 = topzip17.dropna(axis=0,how='any')
norm_17 = norm_17[['zipcode','norm_count','price']]
norm_17.to_csv('norm17.csv')
norm_17

,zipcode,norm_count,price
0,10018,0.034347,229.126949
1,10012,0.024687,224.851354
2,10036,0.024212,208.302691
3,11222,0.019245,120.881401
4,10014,0.017970,227.079401
5,11238,0.017866,130.694239
6,11211,0.017666,146.601759
7,10019,0.016930,200.083219
8,11216,0.016538,107.068129
9,10001,0.016055,213.537485
